In [1]:
import getpass

user = getpass.getuser()
from omni.isaac.kit import SimulationApp

# Set the path below to your desired nucleus server
# Make sure you installed a local nucleus server before this
# simulation_app = SimulationApp({"livesync_usd": f"omniverse://localhost/Users/{user}/demo_raycast.usd"})
# simulation_app = SimulationApp({"livesync_usd": f"omniverse://localhost/Users/{user}/pre-world.usd"})
# simulation_app = SimulationApp({"livesync_usd": f"omniverse://localhost/Users/{user}/texture_sandbox.usd"})
simulation_app = SimulationApp({"livesync_usd": f"omniverse://localhost/Users/{user}/power_thermal_demo.usd"})

[Warning] [omni.isaac.kit] 
[Warning] [omni.isaac.kit] =========================== DEPRECATION WARNING ===========================
[Warning] [omni.isaac.kit] 
[Warning] [omni.isaac.kit] Traceback (most recent call last):
[Warning] [omni.isaac.kit] File '/home/ljb/.local/share/ov/pkg/isaac-sim-4.1.0/kit/python/lib/python3.10/runpy.py', line 196
[Warning] [omni.isaac.kit]   return _run_code(code, main_globals, None,
[Warning] [omni.isaac.kit] File '/home/ljb/.local/share/ov/pkg/isaac-sim-4.1.0/kit/python/lib/python3.10/runpy.py', line 86
[Warning] [omni.isaac.kit]   exec(code, run_globals)
[Warning] [omni.isaac.kit] File '/home/ljb/.local/share/ov/pkg/isaac-sim-4.1.0/kit/python/lib/python3.10/site-packages/ipykernel_launcher.py', line 18
[Warning] [omni.isaac.kit]   app.launch_new_instance()
[Warning] [omni.isaac.kit] File '/home/ljb/.local/share/ov/pkg/isaac-sim-4.1.0/kit/python/lib/python3.10/site-packages/traitlets/config/application.py', line 1075
[Warning] [omni.isaac.kit]   app.sta

it.widget.filter-1.1.4] startup
[1.499s] [ext: omni.graph.core-2.170.3] startup
[1.502s] [ext: omni.kit.window.extensions-1.4.9] startup
[1.507s] [ext: omni.kit.hotkeys.core-1.3.3] startup
[1.508s] [ext: omni.kit.context_menu-1.8.0] startup
[1.509s] [ext: omni.mdl-52.0.1] startup
[1.525s] [ext: omni.iray.libs-0.0.0] startup
[1.529s] [ext: omni.kit.primitive.mesh-1.0.16] startup
[1.532s] [ext: omni.kit.stage_template.core-1.1.21] startup
[1.533s] [ext: omni.mdl.neuraylib-0.2.5] startup
[1.534s] [ext: omni.ujitso.processor.texture-1.0.0] startup
[1.535s] [ext: omni.kit.window.file_exporter-1.0.29] startup
[1.536s] [ext: omni.kit.stage_templates-1.2.3] startup
[1.538s] [ext: omni.volume-0.5.0] startup
[1.540s] [ext: omni.ujitso.client-0.0.0] startup
[1.540s] [ext: omni.kit.window.file_importer-1.1.11] startup
[1.541s] [ext: omni.kit.window.drop_support-1.0.2] startup
[1.542s] [ext: omni.kit.window.file-1.3.52] startup
[1.544s] [ext: omni.kit.window.content_browser_registry-0.0.6] startup


Warp 1.2.1 initialized:
   CUDA Toolkit 11.8, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 3060" (12 GiB, sm_86, mempool enabled)
   Kernel cache:
     /home/ljb/.cache/warp/1.2.1
2.568s] [ext: omni.ui_query-1.1.2] startup
[2.570s] [ext: omni.fabric.commands-1.1.4] startup
[2.576s] [ext: omni.physx.vehicle-106.0.20] startup
[2.593s] [ext: omni.kit.manipulator.prim.core-107.0.3] startup
[2.601s] [ext: omni.kit.graph.editor.core-1.5.3] startup
[2.604s] [ext: omni.kit.ui_test-1.2.18] startup
[2.606s] [ext: omni.kit.graph.usd.commands-1.3.1] startup
[2.608s] [ext: omni.physx.camera-106.0.20] startup
[2.618s] [ext: omni.kit.manipulator.prim.fabric-106.0.1] startup
[2.621s] [ext: omni.kit.manipulator.prim.usd-106.0.1] startup
[2.623s] [ext: omni.physx.cct-106.0.20] startup
[2.634s] [ext: omni.physics.tensors-106.0.20] startup
[2.646s] [ext: omni.isaac.version-1.1.0] startup
[2.648s] [ext: omni.kit.manipulator.selection-104.0.9] startup
[2.652s] [ex

In [2]:
import carb
from omni.physx import get_physx_scene_query_interface
from omni.isaac.core import World
from omni.isaac.core.objects import DynamicCuboid, DynamicSphere, DynamicCone
from omni.timeline import get_timeline_interface
import numpy as np
import matplotlib.pyplot as plt
import omni.ui
from pxr import Kind, Sdf, Usd, UsdGeom, UsdShade
from omni.isaac.core.utils.stage import get_current_stage
import omni.isaac.core.utils.prims as prims_utils

In [3]:
world = World(stage_units_in_meters=1.0)
stage = get_current_stage()

In [4]:
world.clear()
# world.scene.add_default_ground_plane()
# A render/ step or an update call is needed to reflect the changes to the opened USD in Isaac Sim GUI
#  Note: avoid pressing play/ pause or stop in the GUI in this workflow.
world.render()

In [5]:
# Create the Sun
sun = prims_utils.create_prim(
    "/World/Sun",
    "SphereLight",
    position=np.array([1.0, 1.0, 1.0]),
    attributes={
        "inputs:radius": 1,
        "inputs:intensity": 5e3,
        "inputs:color": (1.0, 1.0, 1.0)
    }
)
world.render()

In [6]:
def adjust_light(stage, position):
    lights = []
    for prim in stage.Traverse():
        if prim.GetTypeName() == "SphereLight":
            lights.append(prim)
    if lights:
        light = lights[0]
        light_path = light.GetPath()
        # Adjust position
        translate_attr = stage.GetPrimAtPath(light_path).GetAttribute("xformOp:translate")
        translate_attr.Set(position)
        # Adjust intensity
        intensity_attr = stage.GetPrimAtPath(light_path).GetAttribute("inputs:intensity")
        intensity_attr.Set(3e9)
        # Adjust radius
        radius_attr = stage.GetPrimAtPath(light_path).GetAttribute("inputs:radius")
        radius_attr.Set(.01)

In [7]:
# Define Sun position
# sun_position = (5, 5, 5)
# sun_position = (-5, -5, 5)
sun_position = (5, -5, 5)
# sun_position = (-5, 5, 5)

adjust_light(stage, sun_position)
world.render()

In [8]:
# Load a usd prepared with the meshes we need
from omni.isaac.core.utils.prims import create_prim
from omni.isaac.core.prims.xform_prim import XFormPrim


usd_path = "../../../lunar_base.usd"
prim_path = "/World/Lunar_Base"

ground_path = prim_path + "/Ground"
panel_path = prim_path + "/Solar_Generator/Panel"
support1_path = prim_path + "/Solar_Generator/Support1"
support2_path = prim_path + "/Solar_Generator/Support2"
all_paths = [ground_path, panel_path, support1_path, support2_path]

In [9]:
prim = create_prim(prim_path=prim_path, usd_path=usd_path)
xform_prim = XFormPrim(prim.GetPath().pathString)
world.render()

In [12]:
for path in all_paths:
    prims_utils.set_prim_attribute_value(path, "primvars:displayColor", face_colors)
world.render()

## Utils

In [11]:
# each cube face is a different color:
n_points = len(prims_utils.get_prim_attribute_value(ground_path, "points"))

import matplotlib.colors as mcolors
# Define six equally spaced hues in the HSV space
hues = np.linspace(0, 1, 6, endpoint=False)  # hue = colorhweel: hue 0 and hue 1 are the same so do not include endpoint
rgb_colors = [mcolors.hsv_to_rgb((hue, 1.0, 1.0)) for hue in hues]
 
face_colors = np.zeros((n_points, 3))

# Define the index ranges based on the mesh point structure
assert n_points >= 280
cube_mask = [
    np.arange(0, 100),   # front
    np.arange(100, 200), # back
    np.arange(200, 220), # four sides
    np.arange(220, 240),
    np.arange(240, 260),
    np.arange(260, 280)
]

for i, mask in enumerate(cube_mask):
    face_colors[mask] = rgb_colors[i]